In [ ]:
!pip install https://huggingface.co/opennyaiorg/en_legal_ner_trf/resolve/main/en_legal_ner_trf-any-py3-none-any.whl
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl

In [ ]:
cd '/content/drive/MyDrive/NLP/NLP project/legal_NER-main'

In [ ]:
!pip install -r requirements.txt

In [2]:
import pandas as pd

with open('/content/NER_DEV_PREAMBLE.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('csvfile.csv', encoding='utf-8', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df

,id,annotations,data,meta
0,1ea9dd0b69b64720851a2234a689082f,"[{'result': [{'value': {'start': 17, 'end': 49...",{'text': '$~40 * In The High Court Of Delhi...,{'source': 'motorvehicles_delhi_high_court pre...
1,dc971f32971b4a52a0698650111e8fc3,"[{'result': [{'value': {'start': 117, 'end': 1...",{'text': '1 ...,{'source': 'tax_supremecourts preamble https:/...
2,20026855d8e04d91bde343dd203fec4b,"[{'result': [{'value': {'start': 91, 'end': 12...",{'text': 'R/Scr.A/9089/2017 ...,{'source': 'constitution_gujarat_high_court pr...
3,a3ff6117512d4897846f7e0bd4e46ee7,"[{'result': [{'value': {'start': 0, 'end': 85,...",{'text': 'High Court Of Judicature For Rajasth...,{'source': 'civil_rajasthan_high_court preambl...
4,8664b65997ef4742b697bd03159b5ed1,"[{'result': [{'value': {'start': 47, 'end': 81...",{'text': '1 ...,{'source': 'tax_madras_high_court preamble htt...
...,...,...,...,...
120,d5ad198089c24ae5bdf026cfd57d78c0,"[{'result': [{'value': {'start': 66, 'end': 10...",{'text': 'Carbp-196 of 2016.odt rrpillai ...,{'source': 'civil_bombay_high_court preamble h...
121,2c507d06e7824b669bc095915d4e79fc,"[{'result': [{'value': {'start': 78, 'end': 11...",{'text': 'dss 1 J...,{'source': 'criminal_bombay_high_court preambl...
122,9b9c02d46b5442aeab6a8bfc6d03bebb,"[{'result': [{'value': {'start': 7, 'end': 40,...",{'text': 'In The High Court Of Kerala At Ernak...,{'source': 'tax_kerala_high_court preamble htt...
123,627aad947d0f4413926b9c7c330fa1c1,"[{'result': [{'value': {'start': 0, 'end': 52,...",{'text': 'High Court Of Judicature At Allahaba...,{'source': 'criminal_allahabad_high_court prea...


In [4]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


## Text Preprocessing

### Preproceesing annotations start

In [5]:
annotations_column = df['annotations']

In [6]:
annotations_column =  annotations_column.to_numpy()

In [7]:
annotation_start = []
annotation_end = []


for sent in annotations_column:
  sent_start = []
  sent_end = []
  sent_text = []
  sent_label = []
  for sent_res in sent:
    for res_value in sent_res['result']:
      sent_start.append(res_value['value']['start'])

    for res_value in sent_res['result']:
      sent_end.append(res_value['value']['end'])
    


  annotation_start.append(sent_start)
  annotation_end.append(sent_end)


In [8]:
annotation_start[0]

[17, 199, 286, 301, 323, 361, 436, 458, 578, 677, 705, 802, 816]

In [9]:
len(annotation_end)

125

In [10]:
annotation_text = []
annotation_label = []
for sent in annotations_column:
    sent_text = []
    sent_label = []
    for sent_res in sent:

        for res_value in sent_res['result']:
          sent_text.append(res_value['value']['text'])
        
        for res_value in sent_res['result']:
          sent_label.append(res_value['value']['labels']) 

    annotation_text.append(sent_text)
    annotation_label.append(sent_label)

In [11]:
annotation_text[0]

['High Court Of Delhi At New Delhi',
 'Oriental Insurance Co Ltd.',
 'S.P. Jain',
 'Himanshu Gambhir',
 'Nar\n                    Singh',
 'Pushkar Singh Kanwal',
 'Zaixhu Xie',
 'Qualcomm India Pvt Ltd',
 'Arvind Chaudhary',
 'Ram Kawar',
 'Amit Kumar',
 'Najmi Waziri',
 'Najmi Waziri']

In [12]:
annotation_label[0]

[['COURT'],
 ['PETITIONER'],
 ['LAWYER'],
 ['LAWYER'],
 ['LAWYER'],
 ['LAWYER'],
 ['RESPONDENT'],
 ['RESPONDENT'],
 ['LAWYER'],
 ['LAWYER'],
 ['LAWYER'],
 ['JUDGE'],
 ['JUDGE']]

In [13]:
column_names = ['start', 'end' , 'text' , 'labels']

final_df = pd.DataFrame()
final_df['start'] = annotation_start
final_df['end'] = annotation_end
final_df['text'] = annotation_text
final_df['labels'] = annotation_label

final_df.head()

,start,end,text,labels
0,"[17, 199, 286, 301, 323, 361, 436, 458, 578, 6...","[49, 225, 295, 317, 352, 381, 446, 480, 594, 6...","[High Court Of Delhi At New Delhi, Oriental In...","[[COURT], [PETITIONER], [LAWYER], [LAWYER], [L..."
1,"[117, 328, 451, 701, 824, 1011, 1130]","[139, 354, 467, 726, 851, 1039, 1146]","[Supreme Court Of India, Parmar Samantsinh Ume...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO..."
2,"[91, 236, 938, 1038, 1149, 1190, 1329]","[125, 249, 968, 1054, 1158, 1202, 1342]","[High Court Of Gujarat At Ahmedabad, J.B.Pardi...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],..."
3,"[0, 163, 716, 752, 812]","[85, 187, 734, 794, 846]",[High Court Of Judicature For Rajasthan Bench ...,"[[COURT], [PETITIONER], [RESPONDENT], [RESPOND..."
4,"[47, 275, 402, 728, 888, 982, 1054, 1146, 1324...","[81, 286, 420, 736, 951, 1027, 1115, 1293, 133...","[High Court Of Judicature At Madras, S.Manikum...","[[COURT], [JUDGE], [JUDGE], [PETITIONER], [RES..."


In [14]:
for a,b,c,d in zip(annotation_start[:2], annotation_end[:2],annotation_text[:2],annotation_label[:2]):
  print(a,b,c,d)

[17, 199, 286, 301, 323, 361, 436, 458, 578, 677, 705, 802, 816] [49, 225, 295, 317, 352, 381, 446, 480, 594, 686, 715, 814, 828] ['High Court Of Delhi At New Delhi', 'Oriental Insurance Co Ltd.', 'S.P. Jain', 'Himanshu Gambhir', 'Nar\n                    Singh', 'Pushkar Singh Kanwal', 'Zaixhu Xie', 'Qualcomm India Pvt Ltd', 'Arvind Chaudhary', 'Ram Kawar', 'Amit Kumar', 'Najmi Waziri', 'Najmi Waziri'] [['COURT'], ['PETITIONER'], ['LAWYER'], ['LAWYER'], ['LAWYER'], ['LAWYER'], ['RESPONDENT'], ['RESPONDENT'], ['LAWYER'], ['LAWYER'], ['LAWYER'], ['JUDGE'], ['JUDGE']]
[117, 328, 451, 701, 824, 1011, 1130] [139, 354, 467, 726, 851, 1039, 1146] ['Supreme Court Of India', 'Parmar Samantsinh Umedsinh', 'State Of Gujarat', 'State Election Commission', 'Virendrasinh Mafaji Vaghela', 'Narendra Kumar Ambalal Ravat', 'State Of Gujarat'] [['COURT'], ['PETITIONER'], ['RESPONDENT'], ['PETITIONER'], ['RESPONDENT'], ['PETITIONER'], ['RESPONDENT']]


In [15]:
zip(annotation_start[0], annotation_end[0])

In [16]:
pwd

'/content'

In [17]:
final_df.to_csv('Processed_DataFrame_Dev')

## IOB tagging

In [18]:
import pandas as pd

df_ = pd.read_csv('/content/Processed_DataFrame_Dev')
df_.head()

,Unnamed: 0,start,end,text,labels
0,0,"[17, 199, 286, 301, 323, 361, 436, 458, 578, 6...","[49, 225, 295, 317, 352, 381, 446, 480, 594, 6...","['High Court Of Delhi At New Delhi', 'Oriental...","[['COURT'], ['PETITIONER'], ['LAWYER'], ['LAWY..."
1,1,"[117, 328, 451, 701, 824, 1011, 1130]","[139, 354, 467, 726, 851, 1039, 1146]","['Supreme Court Of India', 'Parmar Samantsinh ...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['..."
2,2,"[91, 236, 938, 1038, 1149, 1190, 1329]","[125, 249, 968, 1054, 1158, 1202, 1342]","['High Court Of Gujarat At Ahmedabad', 'J.B.Pa...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['RESPO..."
3,3,"[0, 163, 716, 752, 812]","[85, 187, 734, 794, 846]",['High Court Of Judicature For Rajasthan Bench...,"[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['..."
4,4,"[47, 275, 402, 728, 888, 982, 1054, 1146, 1324...","[81, 286, 420, 736, 951, 1027, 1115, 1293, 133...","['High Court Of Judicature At Madras', 'S.Mani...","[['COURT'], ['JUDGE'], ['JUDGE'], ['PETITIONER..."


In [19]:
text = df.iloc[[0]]['data']

In [20]:
text[0]['text']

"$~40\n*    In The High Court Of Delhi At New Delhi\n\n%                                                  Decided on: 31.07.2019\n\n+ Mac.App. 976/2018 & Cm Nos. 46122/2018, 15243/2019, 34195/2019\n\n       Oriental Insurance Co Ltd.                     ..... Appellant\n           Through: Mr. S.P. Jain, Mr. Himanshu Gambhir, Mr. Nar\n                    Singh and Mr. Pushkar Singh Kanwal, Advocates.\n\n                          Versus\n\n       Zaixhu Xie & Ors (M/S Qualcomm India Pvt Ltd )\n                                                     ..... Respondents\n           Through: Mr. Arvind Chaudhary, Advocate for Respondent\n                     Nos. 1& 2.\n                     Mr. Ram Kawar, Advocate for Mr. Amit Kumar\n                     Gupta, Advocate for Respondent No.4.\n\nCoram:\nHon'Ble Mr. Justice Najmi Waziri\n\nNajmi Waziri, J. (Oral)\n\n\n"

In [21]:
import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(text[0]['text'])

for d in doc:
  print(d.text)

$
~40


*
   
In
The
High
Court
Of
Delhi
At
New
Delhi



%
                                                 
Decided
on
:
31.07.2019



+
Mac
.
App
.
976/2018
&
Cm
Nos
.
46122/2018
,
15243/2019
,
34195/2019


       
Oriental
Insurance
Co
Ltd.
                    
.....
Appellant

           
Through
:
Mr.
S.P.
Jain
,
Mr.
Himanshu
Gambhir
,
Mr.
Nar

                    
Singh
and
Mr.
Pushkar
Singh
Kanwal
,
Advocates
.


                          
Versus


       
Zaixhu
Xie
&
Ors
(
M
/
S
Qualcomm
India
Pvt
Ltd
)

                                                     
.....
Respondents

           
Through
:
Mr.
Arvind
Chaudhary
,
Advocate
for
Respondent

                     
Nos
.
1
&
2
.

                     
Mr.
Ram
Kawar
,
Advocate
for
Mr.
Amit
Kumar

                     
Gupta
,
Advocate
for
Respondent
No.4
.



Coram
:


Hon'Ble
Mr.
Justice
Najmi
Waziri



Najmi
Waziri
,
J.
(
Oral
)






In [22]:
import re
t = re.sub('\n', " ", text[0]['text'])


In [23]:
" ".join(t.split())

"$~40 * In The High Court Of Delhi At New Delhi % Decided on: 31.07.2019 + Mac.App. 976/2018 & Cm Nos. 46122/2018, 15243/2019, 34195/2019 Oriental Insurance Co Ltd. ..... Appellant Through: Mr. S.P. Jain, Mr. Himanshu Gambhir, Mr. Nar Singh and Mr. Pushkar Singh Kanwal, Advocates. Versus Zaixhu Xie & Ors (M/S Qualcomm India Pvt Ltd ) ..... Respondents Through: Mr. Arvind Chaudhary, Advocate for Respondent Nos. 1& 2. Mr. Ram Kawar, Advocate for Mr. Amit Kumar Gupta, Advocate for Respondent No.4. Coram: Hon'Ble Mr. Justice Najmi Waziri Najmi Waziri, J. (Oral)"

In [24]:
df['data']

0      {'text': '$~40
*    In The High Court Of Delhi...
1      {'text': '1

                                 ...
2      {'text': 'R/Scr.A/9089/2017                   ...
3      {'text': 'High Court Of Judicature For Rajasth...
4      {'text': '1

                                 ...
                             ...                        
120    {'text': 'Carbp-196 of 2016.odt


rrpillai
   ...
121    {'text': 'dss                            1   J...
122    {'text': 'In The High Court Of Kerala At Ernak...
123    {'text': 'High Court Of Judicature At Allahaba...
124    {'text': 'High Court Of Judicature At Allahaba...
Name: data, Length: 125, dtype: object

In [25]:
def preprocess(sent):
  sent = re.sub('\n', " ", sent)
  sent = sent.replace("...", " ")
  sent = " ".join(sent.split())

  return sent


clean_text = []
for i in df['data']:
  clean_text.append(preprocess(i['text']))

In [26]:
len(clean_text)

125

In [27]:
df_['clean_text'] = clean_text

In [28]:
df_.head()

,Unnamed: 0,start,end,text,labels,clean_text
0,0,"[17, 199, 286, 301, 323, 361, 436, 458, 578, 6...","[49, 225, 295, 317, 352, 381, 446, 480, 594, 6...","['High Court Of Delhi At New Delhi', 'Oriental...","[['COURT'], ['PETITIONER'], ['LAWYER'], ['LAWY...",$~40 * In The High Court Of Delhi At New Delhi...
1,1,"[117, 328, 451, 701, 824, 1011, 1130]","[139, 354, 467, 726, 851, 1039, 1146]","['Supreme Court Of India', 'Parmar Samantsinh ...","[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['...",1 Reportable In The Supreme Court Of India Civ...
2,2,"[91, 236, 938, 1038, 1149, 1190, 1329]","[125, 249, 968, 1054, 1158, 1202, 1342]","['High Court Of Gujarat At Ahmedabad', 'J.B.Pa...","[['COURT'], ['JUDGE'], ['PETITIONER'], ['RESPO...",R/Scr.A/9089/2017 Judgment In The High Court O...
3,3,"[0, 163, 716, 752, 812]","[85, 187, 734, 794, 846]",['High Court Of Judicature For Rajasthan Bench...,"[['COURT'], ['PETITIONER'], ['RESPONDENT'], ['...",High Court Of Judicature For Rajasthan Bench A...
4,4,"[47, 275, 402, 728, 888, 982, 1054, 1146, 1324...","[81, 286, 420, 736, 951, 1027, 1115, 1293, 133...","['High Court Of Judicature At Madras', 'S.Mani...","[['COURT'], ['JUDGE'], ['JUDGE'], ['PETITIONER...",1 In The High Court Of Judicature At Madras Da...


In [29]:
s = df_.iloc[0]['clean_text']

for i in range(7,40):
  print(s[i])

I
n
 
T
h
e
 
H
i
g
h
 
C
o
u
r
t
 
O
f
 
D
e
l
h
i
 
A
t
 
N
e
w


In [30]:
df_.iloc[1]['clean_text']

import spacy
nlp = spacy.load('en_core_web_sm')

doc = nlp(df_.iloc[0]['clean_text'])

print(doc[0])

$


In [31]:
span = doc.char_span(73,100,label = "COURT")
span

## New solution

In [32]:
def sentence_cleaner(sentence):
    '''
    replacing the unwanted charachters
    '''
    reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
    return " ".join(reg.findall(sentence.replace('\n', ' '))).replace(" / ", '/')

In [33]:
def sentence_cleaner_v1(sentence):
    '''
    replacing the unwanted charachters
    '''
    sent_1 = " ".join(sentence.replace("\n" , " ").replace("..." , " ").replace("-", " ").replace("," , " ").replace(":" , " ").replace("=",  " ").split())
    doc = nlp(sent_1)
    doc_list = []
    for i in doc:
      doc_list.append(str(i))
    return doc_list

In [34]:
def sentence_tagging(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,temp_len)
    if temp_len > 1 :
        for i in range(temp_len):
            #print(i, temp_len - 1)
            if i == 0 :
              word2ent[temp_sent[i]] =  "B-" + ann['value']['labels'][0]
            
            elif i == (temp_len - 1):
              word2ent[temp_sent[i]] =  "E-" + ann['value']['labels'][0]

            else :
              word2ent[temp_sent[i]] =  "I-" + ann['value']['labels'][0]

    else :
        word2ent[temp_sent[0]] = "B-" + ann['value']['labels'][0]

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys() and word + ',' not in word2ent.keys() :
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
        tag_list.append(word2ent[word])
      
      else :
          tag_list.append(word2ent[word])
  
  
  print(word2ent )
  return tag_list
      

In [35]:
def sentence_tagging_v1(sent, annotation):

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = (sent[ ann['value']['start']: ann['value']['end']]).split()
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner(sent)
  sent = sent.split()
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        
        word2ent[word] = "O"
        #print(word , word2ent[word])
      tag_list.append(word2ent[word])
      
  
  
  #print(word2ent )
  return tag_list
      

In [37]:
def sentence_tagging_v2(sent, annotation):

  """
  current using word
  """

  #creating dictonary to store word to entity
  word2ent = {}
  sent = sent['text']
  for ann in (annotation[0]['result']):
    temp_sent = sentence_cleaner_v1(sent[ ann['value']['start']: ann['value']['end']])
    
    #Customized IOBE tagging

    temp_len = len(temp_sent)
    #print(temp_sent,word2ent)
    for i in range(temp_len):
        #print(i, temp_len - 1)
        #assign empty list if word not present in dictonary
        if temp_sent[i] not in word2ent:
            word2ent[temp_sent[i]] = []

        if i == 0 :
          word2ent[temp_sent[i]].append("B-" + ann['value']['labels'][0])
        
        elif i == (temp_len - 1):
          word2ent[temp_sent[i]].append("E-" + ann['value']['labels'][0])

        else :
          word2ent[temp_sent[i]].append("I-" + ann['value']['labels'][0])

  #adding 'O' tag
  sent = sentence_cleaner_v1(sent)
  tag_list = []
  for word in sent:
      #print(word)
      if word not in word2ent.keys():
        tag = "O"
      else :
        if len(word2ent[word]) != 0 :
          tag = word2ent[word].pop(0)
        else :
          tag = "O"

      #print(word,tag)   
      tag_list.append(str(tag))
      
  
  
  #print(word2ent )
  return tag_list
      

In [38]:
IOBE_tagging = []
i = 0
for sent, anno in zip( df['data'].to_list(), df['annotations'].to_list()):
    IOBE_tagging.append(sentence_tagging_v2(sent,anno))
    
  

In [39]:
IOBE_tagging[2]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-COURT',
 'I-COURT',
 'I-COURT',
 'I-COURT',
 'I-COURT',
 'E-COURT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-JUDGE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-PETITIONER',
 'I-PETITIONER',
 'E-PETITIONER',
 'O',
 'O',
 'O',
 'B-RESPONDENT',
 'I-RESPONDENT',
 'E-RESPONDENT',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LAWYER',
 'I-LAWYER',
 'E-LAWYER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LAWYER',
 'E-LAWYER',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',

In [40]:
final_df['tags'] = IOBE_tagging
final_df.head()

,start,end,text,labels,tags
0,"[17, 199, 286, 301, 323, 361, 436, 458, 578, 6...","[49, 225, 295, 317, 352, 381, 446, 480, 594, 6...","[High Court Of Delhi At New Delhi, Oriental In...","[[COURT], [PETITIONER], [LAWYER], [LAWYER], [L...","[O, O, O, O, O, B-COURT, I-COURT, I-COURT, I-C..."
1,"[117, 328, 451, 701, 824, 1011, 1130]","[139, 354, 467, 726, 851, 1039, 1146]","[Supreme Court Of India, Parmar Samantsinh Ume...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO...","[O, O, O, O, B-COURT, I-COURT, I-COURT, E-COUR..."
2,"[91, 236, 938, 1038, 1149, 1190, 1329]","[125, 249, 968, 1054, 1158, 1202, 1342]","[High Court Of Gujarat At Ahmedabad, J.B.Pardi...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],...","[O, O, O, O, O, O, O, O, B-COURT, I-COURT, I-C..."
3,"[0, 163, 716, 752, 812]","[85, 187, 734, 794, 846]",[High Court Of Judicature For Rajasthan Bench ...,"[[COURT], [PETITIONER], [RESPONDENT], [RESPOND...","[B-COURT, I-COURT, I-COURT, I-COURT, I-COURT, ..."
4,"[47, 275, 402, 728, 888, 982, 1054, 1146, 1324...","[81, 286, 420, 736, 951, 1027, 1115, 1293, 133...","[High Court Of Judicature At Madras, S.Manikum...","[[COURT], [JUDGE], [JUDGE], [PETITIONER], [RES...","[O, O, O, B-COURT, I-COURT, I-COURT, I-COURT, ..."


In [41]:
sent_1 = df['data'].to_list()[2]['text']
sent_1

'R/Scr.A/9089/2017                                           Judgment\n\n\n\n\n           In The High Court Of Gujarat At Ahmedabad\n\n           Special Criminal Application No. 9089 of 2017\n\n\nFor Approval And Signature:\nHonourable Mr.Justice J.B.Pardiwala                                Sd/-\n\n==========================================================\n\n1     Whether Reporters of Local Papers may be allowed to see the              Yes\n      judgment ?\n\n2     To be referred to the Reporter or not ?                                  Yes\n\n3     Whether their Lordships wish to see the fair copy of the                 No\n      judgment ?\n\n4     Whether this case involves a substantial question of law as to           No\n      the interpretation of the Constitution of India or any order made\n      thereunder ?\n\n      Circulate this Judgment in the Subordinate Judiciary.\n\n==========================================================\n            Harishankar Gayaprasad Jaiswal.

In [ ]:
# tag_2 = final_df['tags'].to_list()[77]
# tag_2

In [42]:
# [\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]

reg = re.compile("[\w]*[\w.]*\w[\w']*\w[\w\/]*|[\w\/O]")
" ".join(reg.findall(sent_1.replace('\n', ' '))).replace(" / ", '/')

'R/Scr A/9089/2017 Judgment In The High Court Of Gujarat At Ahmedabad Special Criminal Application No 9089 of 2017 For Approval And Signature Honourable Mr.Justice J.B.Pardiwala Sd/ 1 Whether Reporters of Local Papers may be allowed to see the Yes judgment 2 To be referred to the Reporter or not Yes 3 Whether their Lordships wish to see the fair copy of the No judgment 4 Whether this case involves a substantial question of law as to No the interpretation of the Constitution of India or any order made thereunder Circulate this Judgment in the Subordinate Judiciary Harishankar Gayaprasad Jaiswal Petitioner Versus State Of Gujarat 3 Respondents Appearance Mr H N Sevak for the PETITIONER s No 1 Ms Moxa Thakkar App for the RESPONDENT s No 1 Coram Honourable Mr.Justice J.B.Pardiwala Date 08/03/2018 Oral Judgment'

In [43]:
" ".join(sent_1.replace(" "," ").split())

'R/Scr.A/9089/2017 Judgment In The High Court Of Gujarat At Ahmedabad Special Criminal Application No. 9089 of 2017 For Approval And Signature: Honourable Mr.Justice J.B.Pardiwala Sd/- ========================================================== 1 Whether Reporters of Local Papers may be allowed to see the Yes judgment ? 2 To be referred to the Reporter or not ? Yes 3 Whether their Lordships wish to see the fair copy of the No judgment ? 4 Whether this case involves a substantial question of law as to No the interpretation of the Constitution of India or any order made thereunder ? Circulate this Judgment in the Subordinate Judiciary. ========================================================== Harishankar Gayaprasad Jaiswal.... Petitioner Versus State Of Gujarat & 3.... Respondents ========================================================== Appearance: Mr H N Sevak for the PETITIONER(s) No. 1 Ms Moxa Thakkar, App for the RESPONDENT(s) No. 1 =================================================

In [44]:
word2count = {}

for sent in df['data'].to_list():
    for word in sent['text'].split():
        if word not in word2count:
          word2count[word] = 0
        else:
          word2count[word] += 1

In [45]:
for key in word2count.keys():
  if re.search(r'\bM+\b', key):
    print(key)

(M/S
M
S.M.Subramaniam
M.R.Anitha
M.
M/s.
M.Seetharama
M/S.
M/s
M.P.
Crl.M.C.2663/2021
M/S
Crl.M.C.2730/2021
M.A.K.
S.M.
X.A.C.M.M.
(O&M)
M.K.
M.Govindaraj
M.D,
M/S.Siraj
Crl.M.C.,
Crl.M.A.No.1/2019
Crl.M.C.No.1797/2017.
Crl.M.C.No.1797
2019(O&M)
M.G.Road
M.Bapi
W.M.P.Nos.17661
2.M/s.Covai
W.M.P.Nos.10241
Crl.M.(Bail)
M.P.Chothy,
M.P.Chothy
C.M.
M.Venugopal
C.M.A.No.2249
M.P.No.1
M/s.Row
M.R.
V.M.
V.M.Velumani
C.M.A(MD)No.864
M.C.O.P.No.830
M.D.
M.K.Geore,
M.T.Issac,
Pandey,S.M.
K.M.


## Using Tokenizer

In [46]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()

In [47]:
sent_1 = " ".join(sent_1.replace("\n" , " ").replace("..." , " ").replace("-", "").split())
doc = nlp(sent_1)
doc_list = []
for i in doc:
  doc_list.append(str(i))
doc_list

['R',
 '/',
 'Scr',
 '.',
 'A/9089/2017',
 'Judgment',
 'In',
 'The',
 'High',
 'Court',
 'Of',
 'Gujarat',
 'At',
 'Ahmedabad',
 'Special',
 'Criminal',
 'Application',
 'No',
 '.',
 '9089',
 'of',
 '2017',
 'For',
 'Approval',
 'And',
 'Signature',
 ':',
 'Honourable',
 'Mr.',
 'Justice',
 'J.B.Pardiwala',
 'Sd/',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '1',
 'Whether',
 'Reporters',
 'of',
 'Local',
 'Papers',
 'may',
 'be',
 'allowed',
 'to',
 'see',
 'the',
 'Yes',
 'judgment',
 '?',
 '2',
 'To',
 'be',
 'referred',
 'to',
 'the',
 'Reporter',
 'or',
 'not',
 '?',
 'Yes',
 '3',
 'Whether',
 'their',
 'Lordships',
 'wish',
 'to',
 'see',
 'the',
 'fair',
 'copy',
 'of',
 'the',


# Clean the sentences

In [48]:
clean_sentences = []

for sent in df['data'].to_list():
    clean_sentences.append(sentence_cleaner_v1(sent['text']))



In [49]:
final_df['clean_sent'] = clean_sentences
final_df.head()

,start,end,text,labels,tags,clean_sent
0,"[17, 199, 286, 301, 323, 361, 436, 458, 578, 6...","[49, 225, 295, 317, 352, 381, 446, 480, 594, 6...","[High Court Of Delhi At New Delhi, Oriental In...","[[COURT], [PETITIONER], [LAWYER], [LAWYER], [L...","[O, O, O, O, O, B-COURT, I-COURT, I-COURT, I-C...","[$, ~40, *, In, The, High, Court, Of, Delhi, A..."
1,"[117, 328, 451, 701, 824, 1011, 1130]","[139, 354, 467, 726, 851, 1039, 1146]","[Supreme Court Of India, Parmar Samantsinh Ume...","[[COURT], [PETITIONER], [RESPONDENT], [PETITIO...","[O, O, O, O, B-COURT, I-COURT, I-COURT, E-COUR...","[1, Reportable, In, The, Supreme, Court, Of, I..."
2,"[91, 236, 938, 1038, 1149, 1190, 1329]","[125, 249, 968, 1054, 1158, 1202, 1342]","[High Court Of Gujarat At Ahmedabad, J.B.Pardi...","[[COURT], [JUDGE], [PETITIONER], [RESPONDENT],...","[O, O, O, O, O, O, O, O, B-COURT, I-COURT, I-C...","[R, /, Scr, ., A/9089/2017, Judgment, In, The,..."
3,"[0, 163, 716, 752, 812]","[85, 187, 734, 794, 846]",[High Court Of Judicature For Rajasthan Bench ...,"[[COURT], [PETITIONER], [RESPONDENT], [RESPOND...","[B-COURT, I-COURT, I-COURT, I-COURT, I-COURT, ...","[High, Court, Of, Judicature, For, Rajasthan, ..."
4,"[47, 275, 402, 728, 888, 982, 1054, 1146, 1324...","[81, 286, 420, 736, 951, 1027, 1115, 1293, 133...","[High Court Of Judicature At Madras, S.Manikum...","[[COURT], [JUDGE], [JUDGE], [PETITIONER], [RES...","[O, O, O, B-COURT, I-COURT, I-COURT, I-COURT, ...","[1, In, The, High, Court, Of, Judicature, At, ..."


## Word2idx

In [50]:
def get_dict_map(df , token_or_tag):
    """
    converts word to index

    [Need changes !!!]
    """
    tok2idx = {}
    idx2tok = {}

    vocab = []

    if token_or_tag == 'token':
        for sent in df['clean_sent'].to_list():
          vocab.extend( list(set(sent)) )

        vocab = set(vocab)

    else :
        for sent in df['tags'].to_list():
          vocab.extend( list(set(sent)) )
        
        vocab = set(vocab)


    idx2tok = {idx:tok for idx , tok in enumerate(vocab)}
    tok2idx = {tok:idx for idx , tok in enumerate(vocab)}

    return tok2idx, idx2tok



word2idx, idx2word = get_dict_map(final_df, 'token')
# tag2idx, idx2tag = get_dict_map(final_df, 'tag')


In [52]:
import json
 
# Opening JSON file
with open('tag2idx') as tag2idx_file:
    tag2idx = json.load(tag2idx_file)

In [53]:
import json
 
# Opening JSON file
with open('idx2tag') as idx2tag_file:
    idx2tag = json.load(idx2tag_file)

In [ ]:
tag2idx

In [ ]:
idx2tag

In [56]:
"""
Padding and converting from word sequence to index sequence

"""

max_len = max([len(sent) for sent in final_df['clean_sent'].tolist()])
pad_idx = len(word2idx)
tag_pad_idx = tag2idx['O']
word_idx_sequence = []
tag_idx_sequence = []
pad = "PAD"

for sent in final_df['clean_sent'].to_list():
    #print(sent)
    word_idx_sequence.append([word2idx[word] for word in sent] + [pad_idx for _ in range(max_len - len(sent))])


for tag_seq in final_df['tags'].to_list():
    tag_idx_sequence.append([tag2idx[tag] for tag in tag_seq] + [tag_pad_idx for _ in range(max_len - len(tag_seq))])


In [57]:
final_df['word_idx'] = word_idx_sequence
final_df['tag_idx'] = tag_idx_sequence



In [58]:
len(tag2idx)

16

In [59]:
tag2idx

{'E-JUDGE': 0,
 'E-PETITIONER': 1,
 'B-LAWYER': 2,
 'E-RESPONDENT': 3,
 'I-RESPONDENT': 4,
 'O': 5,
 'I-PETITIONER': 6,
 'B-JUDGE': 7,
 'B-PETITIONER': 8,
 'E-LAWYER': 9,
 'I-JUDGE': 10,
 'I-LAWYER': 11,
 'E-COURT': 12,
 'B-COURT': 13,
 'B-RESPONDENT': 14,
 'I-COURT': 15}

In [60]:
word2idx

{'P.C.': 0,
 'A.K.Kishore': 1,
 '1976': 2,
 '19.4.2019': 3,
 'A.C.Vidhya': 4,
 '4099': 5,
 'No.1501': 6,
 'Kottayam': 7,
 'g.doc': 8,
 'calling': 9,
 'Note': 10,
 'P.S.': 11,
 'Kalina': 12,
 'Biswas': 13,
 'Sarathy': 14,
 'Prounounced': 15,
 'Kranti': 16,
 'Murli': 17,
 '151': 18,
 'Parvinder': 19,
 'Application': 20,
 'o': 21,
 'V.': 22,
 'Sonak': 23,
 'owned': 24,
 '1548': 25,
 'H.L.': 26,
 'Sultana': 27,
 '1994(Nmr': 28,
 'Lpa': 29,
 '12876': 30,
 'Pcr': 31,
 'Prithu': 32,
 'Gst': 33,
 'Cargo': 34,
 'Palakkad': 35,
 'institution': 36,
 'Ble': 37,
 '366': 38,
 'Property': 39,
 '792': 40,
 'No.53174/2022': 41,
 'Apoorva': 42,
 'O.Late': 43,
 'No.24481/2012(Mv': 44,
 'Triumph': 45,
 '1545': 46,
 'Noel': 47,
 'Avratosh': 48,
 'decision': 49,
 'Rajshekhar': 50,
 'Royal': 51,
 '11th': 52,
 '119/2018': 53,
 'Theppakulathu': 54,
 'Dhanvantri': 55,
 'Jindal': 56,
 'Summons': 57,
 '340': 58,
 '2021': 59,
 'D': 60,
 'S.B.': 61,
 'B.Pugalendhi': 62,
 'Claims': 63,
 'B.K.': 64,
 'K.M.': 65,
 'At

In [61]:
final_df.to_csv("Final_DataFrame_Dev_Preamble")